# Input names

In [47]:
newSceneName = "AdminMenun"
userHistoryEventSceneStart = f"Перешел к сцене {newSceneName}"
tab = '    '

# Generation

In [48]:
import re

def toCamelCaseFromSnake(snake_str):
    return "".join(x.capitalize() for x in snake_str.lower().split("_"))

def toSnakeCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()

def toKebabCaseFromCamel(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1-\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1-\2', name).lower()


In [49]:
from pathlib import Path
import platform
import codecs

isWindows = platform.system() == 'Windows'

def writeFileData(filePath: Path, content):
    if isWindows:
        data = content
        with codecs.open(filePath, 'w', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('w') as file:
            file.write(data)

def addLinesToFileData(filePath: Path, content):
    if isWindows:
        data = content
        with codecs.open(filePath, 'a', encoding= 'utf-8') as file:
            file.write(data)
    else:
        data = content
        with filePath.open('a') as file:
            file.write(data)

def getFileData(filePath: Path):
    if isWindows:
        with codecs.open(filePath, 'r', 'utf-8') as file:
            content = file.read()
    else:
        with filePath.open() as file:
            content = file.read()
    return content

def replaceInFileData(filePath: Path, placeholder: str, newContent: str):
    fileContent = getFileData(filePath)
    fileContent = fileContent.replace(
        placeholder,
        newContent
    )
    writeFileData(filePath, fileContent)

In [50]:
sceneNameCapitaliseMask = "<#SceneNameCapitalise#>"
sceneNameCamelCaseMask = "<#SceneNameCamelCase#>"
fileContentTemplate = """import { Message, Update } from 'telegraf/typings/core/types/typegram'
import { Markup, Context } from 'telegraf'
import { SceneName } from '../enums/scene-name.enum'
import { SceneHandlerCompletion, Scene, SceneCallbackData } from '../scene.interface'
import { logger } from 'src/app.logger'

// =====================
// Scene data class
// =====================

interface ISceneData {
    
}

export class <#SceneNameCapitalise#>Scene extends Scene<ISceneData> {
    // =====================
    // Properties
    // =====================

    readonly name: SceneName = SceneName.<#SceneNameCamelCase#>
    get dataDefault(): ISceneData {
        return this.generateData({})
    }

    // =====================
    // Public methods
    // =====================

    async handleEnterScene(ctx: Context<Update>): Promise<SceneHandlerCompletion> {
        logger.log(`${this.name} scene handleEnterScene. User: ${ctx.from.id} ${ctx.from.username}`)
        await this.logToUserHistory(this.historyEvent.startScene<#SceneNameCapitalise#>)
        
        await ctx.replyWithHTML(
            `Hello from <b><#SceneNameCapitalise#></b>`,
            super.keyboardMarkupWithAutoLayoutFor(['Hello!'])
        )

        return this.completion.inProgress(
            this.generateData({})
        )
    }

    async handleMessage(ctx: Context<Update>, dataRaw: object): Promise<SceneHandlerCompletion> {
        logger.log(`${this.name} scene handleMessage. User: ${ctx.from.id} ${ctx.from.username}`)

        let data = this.restoreData(dataRaw)
        const message = ctx.message as Message.TextMessage

        await ctx.replyWithHTML(`Goodbye`, Markup.removeKeyboard())

        return this.completion.complete(SceneName.mainMenu)
    }

    async handleCallback(ctx: Context<Update>, dataRaw: SceneCallbackData): Promise<SceneHandlerCompletion> {
        throw Error('Method not implemented.')
    }

    // =====================
    // Private methods
    // =====================
}
"""

## Configuration

In [51]:
sceneNameCapitalise = newSceneName[0].upper() + newSceneName[1:]
sceneNameCamelCase = newSceneName[0].lower() + newSceneName[1:]
sceneNameKebabCase = toKebabCaseFromCamel(sceneNameCapitalise)

In [52]:
# Check folder path

scenesPath = Path("../src/presentation/scenes/implementations/")
creatingSceneFolder = scenesPath
creatingSceneSourceFile = creatingSceneFolder / f"{sceneNameKebabCase}.scene.ts"

userHistoryEventFilePath = Path("../src/core/user/enums/user-history-event.enum.ts")
sceneNameFilePath = Path("../src/presentation/scenes/enums/scene-name.enum.ts")
sceneFactoryFilePath = Path("../src/presentation/scenes/scene-factory.ts")


In [53]:
# Source code names update

fileContent = fileContentTemplate.replace(sceneNameCapitaliseMask, sceneNameCapitalise)
fileContent = fileContent.replace(sceneNameCamelCaseMask, sceneNameCamelCase)

## Source code generation

In [54]:
# creatingSceneFolder.mkdir(parents=True, exist_ok=True)
writeFileData(creatingSceneSourceFile, fileContent)

In [55]:
# UserHistoryEvent
placeholder = '/** New scene event placeholder */'
replaceInFileData(
    filePath=userHistoryEventFilePath,
    placeholder=placeholder,
    newContent=f"startScene{sceneNameCapitalise} = '{userHistoryEventSceneStart}',\n{tab}{placeholder}"
)

# # SceneName
# placeholder = '/** New scene name placeholder */'
# replaceInFileData(
#     filePath=sceneNameFilePath,
#     placeholder=placeholder,
#     newContent=f"{sceneNameCamelCase} = '{sceneNameCamelCase}',\n{tab}{placeholder}"
# )

# # SceneFactory
# placeholder = '/** New scene import placeholder */'
# replaceInFileData(
#     filePath=sceneFactoryFilePath,
#     placeholder=placeholder,
#     newContent="import { " + sceneNameCapitalise + "Scene } from './implementations/" + sceneNameKebabCase + ".scene'\n" + placeholder
# )

# placeholder = '/** New scene generation placeholder */'
# replaceInFileData(
#     filePath=sceneFactoryFilePath,
#     placeholder=placeholder,
#     newContent=f"case SceneName.{sceneNameCamelCase}:\n{tab * 4}return new {sceneNameCapitalise}Scene(configuration)\n{tab * 3}{placeholder}"
# )